In [1]:
import os
import datetime
import pandas as pd
import numpy as np
import soundfile as sf
from matplotlib import pyplot as plt

import tensorflow as tf
import tensorflow_io as tfio
from tensorboard import notebook

import keras.models
from keras import regularizers
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten
from keras.layers import Dropout
from keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import concatenate

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler


from tensorflow.keras.layers import Input, Conv2D
from tensorflow.keras.layers import MaxPool2D, Flatten, Dense, Lambda
from tensorflow.keras import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.xception import Xception

In [2]:
def load_audio(file_name):
    audio_data, sample_rate = sf.read(file_name)
    return audio_data

In [3]:
def preprocess(file_path):
    wav = load_audio(file_path)
    wav = wav[:960000]
    zero_padding = tf.zeros([960000] - tf.shape(wav), dtype=tf.float32)
    wav = tf.concat([zero_padding, wav], 0)

    spectrogram = tfio.audio.spectrogram(
        wav, nfft=892, window=892, stride=int(960000/448) + 1)

    mel_spectrogram = tfio.audio.melscale(
        spectrogram, rate=32000, mels=224, fmin=0, fmax=12000)

    dbscale_mel_spectrogram = tfio.audio.dbscale(
        mel_spectrogram, top_db=80)

    freq_mask = tfio.audio.freq_mask(dbscale_mel_spectrogram, param=4)

    time_mask = tfio.audio.time_mask(freq_mask, param=4)
    time_mask = tf.expand_dims(time_mask, axis=2)
    return time_mask

In [4]:
def create_model(num_labels):
    xception = Xception(input_shape=[448, 224, 3], weights='imagenet', include_top=False)
    
    for layer in xception.layers:
        layer.trainable = False

    # Fully connected layers  
    image = Flatten()(xception.output)
    
    info_input = Input((2,))
    info = Dense(16, activation="relu", kernel_regularizer=regularizers.l2(0.001))(info_input)
    info = Dense(32, activation="relu", kernel_regularizer=regularizers.l2(0.001))(info)
    info = Dense(64, activation="relu", kernel_regularizer=regularizers.l2(0.001))(info)
    info = Dense(128, activation="relu", kernel_regularizer=regularizers.l2(0.001))(info)
    info = Dropout(.5)(info)
    
    combined = concatenate([image, info])
    output = Dense(512, activation="relu", kernel_regularizer=regularizers.l2(0.001))(combined)
    output = Dropout(.5)(output)
    output = Dense(256, activation="relu", kernel_regularizer=regularizers.l2(0.001))(output)
    output = Dense(units = num_labels, activation ='softmax')(output)
    
    model = Model([xception.input, info_input], [output])
    model.summary()
    optimizer = keras.optimizers.Adam()
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=["accuracy"])
    return model

In [5]:
def train_model(trainAttrX, trainImagesX, trainY, testAttrX, testImagesX, testY, num_labels, batch_amt, epoch_amt, save):
    model = create_model(num_labels)

    log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


    model.fit( x=[trainImagesX, trainAttrX], y=trainY, validation_data=([testImagesX, testAttrX], testY),
        epochs=epoch_amt, batch_size=batch_amt, callbacks=[tensorboard_callback], verbose=False)
    
    if save:
        model.save("model")

    return model

In [6]:
def predict_file(file_name, model, label_encoder, latitude, longitude, date):
    image = preprocess(file_name)

    data = [[image, latitude, longitude, date]]
    df = pd.DataFrame(data, columns=['image', 'latitude', 'longitude'])
    continuous = ["latitude", "longitude"]
    
    cs = MinMaxScaler()
    predictContinuous = cs.fit_transform(df[continuous])
    
    predicted_label = model.predict([image.numpy().reshape(1, 224, 224, 3), predictContinuous])
    classes_x = np.argmax(predicted_label, axis=1)
    prediction_class = labelencoder.inverse_transform(classes_x)
    return prediction_class

In [7]:
def process_images(df, inputPath):
    images = []
    for index_num, row in df.iterrows():
        images.append(preprocess(inputPath + row["primary_label"] + "/" + row["filename"]))
    return np.asarray(images).astype(np.float32)

In [8]:
def load_attributes(inputPath):
    cols = ["latitude", "longitude", "filename", "primary_label"]
    df = pd.read_csv(inputPath, skipinitialspace=True, usecols=cols)
    df = df.loc[df['primary_label'] <= "ameavo"]
    df = shuffle(df)
    df.reset_index(inplace=True, drop=True)
    return df

In [9]:
def process_attributes(inputPath, train, test):
    continuous = ["latitude", "longitude"]
    
    cs = MinMaxScaler()
    trainX = cs.fit_transform(train[continuous])
    testX = cs.transform(test[continuous])
    
    return (trainX, testX)

In [10]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [11]:
df = load_attributes("./Data/train_metadata.csv")

In [12]:
images = process_images(df, "./Data/Audio/")

In [13]:
images = np.repeat(images,repeats=3,axis=3)

In [14]:
split = train_test_split(df, images, test_size=0.2, random_state=0)
(trainAttrX, testAttrX, trainImagesX, testImagesX) = split
label_encoder = LabelEncoder()
trainY = to_categorical(label_encoder.fit_transform(trainAttrX["primary_label"]))
testY = to_categorical(label_encoder.fit_transform(testAttrX["primary_label"]))

(trainAttrX, testAttrX) = process_attributes(df,
    trainAttrX, testAttrX)


In [15]:
num_labels = trainY.shape[1]

In [ ]:
train_model(trainAttrX, trainImagesX, trainY, testAttrX, testImagesX, testY, num_labels, 16, 150, True)

#model = keras.models.load_model("model")

#preds = model.predict([testImagesX, testAttrX])

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 448, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 223, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 223, 111, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                             

 2D)                                                                                              
                                                                                                  
 block4_sepconv1_bn (BatchNorma  (None, 56, 28, 728)  2912       ['block4_sepconv1[0][0]']        
 lization)                                                                                        
                                                                                                  
 block4_sepconv2_act (Activatio  (None, 56, 28, 728)  0          ['block4_sepconv1_bn[0][0]']     
 n)                                                                                               
                                                                                                  
 block4_sepconv2 (SeparableConv  (None, 56, 28, 728)  536536     ['block4_sepconv2_act[0][0]']    
 2D)                                                                                              
          

                                                                                                  
 block7_sepconv1_act (Activatio  (None, 28, 14, 728)  0          ['add_4[0][0]']                  
 n)                                                                                               
                                                                                                  
 block7_sepconv1 (SeparableConv  (None, 28, 14, 728)  536536     ['block7_sepconv1_act[0][0]']    
 2D)                                                                                              
                                                                                                  
 block7_sepconv1_bn (BatchNorma  (None, 28, 14, 728)  2912       ['block7_sepconv1[0][0]']        
 lization)                                                                                        
                                                                                                  
 block7_se

 2D)                                                                                              
                                                                                                  
 block9_sepconv3_bn (BatchNorma  (None, 28, 14, 728)  2912       ['block9_sepconv3[0][0]']        
 lization)                                                                                        
                                                                                                  
 add_7 (Add)                    (None, 28, 14, 728)  0           ['block9_sepconv3_bn[0][0]',     
                                                                  'add_6[0][0]']                  
                                                                                                  
 block10_sepconv1_act (Activati  (None, 28, 14, 728)  0          ['add_7[0][0]']                  
 on)                                                                                              
          

 block12_sepconv2_bn (BatchNorm  (None, 28, 14, 728)  2912       ['block12_sepconv2[0][0]']       
 alization)                                                                                       
                                                                                                  
 block12_sepconv3_act (Activati  (None, 28, 14, 728)  0          ['block12_sepconv2_bn[0][0]']    
 on)                                                                                              
                                                                                                  
 block12_sepconv3 (SeparableCon  (None, 28, 14, 728)  536536     ['block12_sepconv3_act[0][0]']   
 v2D)                                                                                             
                                                                                                  
 block12_sepconv3_bn (BatchNorm  (None, 28, 14, 728)  2912       ['block12_sepconv3[0][0]']       
 alization

                                                                                                  
 dense_6 (Dense)                (None, 4)            1028        ['dense_5[0][0]']                
                                                                                                  
Total params: 123,831,356
Trainable params: 102,969,876
Non-trainable params: 20,861,480
__________________________________________________________________________________________________


In [ ]:
prediction = predict_file("./Data/Audio/aldfly/XC477348.ogg", model, label_encoder,
                              42.0458, -79.441, "2019-05-27")
prediction